In [60]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import chromedriver_autoinstaller

import subprocess


from pathlib import Path
from time import sleep
from win10toast import ToastNotifier
from re import search
import copy
import json
import string
import glob
import os
import shutil
import sys
sys.path.append('./')

In [61]:
chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
chrome_ver

'132'

In [62]:
driver_name = chromedriver_autoinstaller.install(True)
driver_name

'c:\\Users\\강찬석\\Desktop\\lemon\\132\\chromedriver.exe'

In [63]:
from selenium.webdriver.chrome.service import Service
service = Service(driver_name)

options = Options()
# # options

options.add_argument('--window-size=1920x1080')
# options.add_experimental_option("debuggerAddress", "127.0.0.1:32999")
# options.add_argument("--disable-gpu")
# options.add_argument("--headless")

driver = webdriver.Chrome(service=service, options=options)


In [64]:
url = "https://wwme.kr/lemon/play?mode=normal"
driver.get(url)

In [221]:
driver.refresh()


start_button = "/html/body/div/div[1]/main/div/main/div[2]/div[2]/div[1]/div/div[2]/button"
click_toggle_button = "/html/body/div/div[1]/main/div/main/div[1]/div/div/button[2]"
music_toggle_button = "/html/body/div/div[1]/main/div/main/div[1]/div/div/button[1]"
table = "/html/body/div/div[1]/main/div/main/div[2]/div[2]/div"
row_size = 10
col_size = 17

driver.find_element(By.XPATH, start_button).click()


driver.find_element(By.XPATH, click_toggle_button).click()
driver.find_element(By.XPATH, music_toggle_button).click()

sleep(3)

In [222]:
def solve_routine(board):
    answers = []

    for i in range(row_size):
        for j in range(col_size):
            # print(f"{i},{j} calcluating")
            box = board[i][j]
            idx = 1
            while box<10 and i+idx<row_size:
                box+=board[i+idx][j]
                idx+=1
            if(box==10):
                answers.append([[i,j], [i+idx-1,j]])
                for r in range(idx):
                    board[i+r][j] = 0
                continue

            box = board[i][j]
            idx = 1
            while box<10 and j+idx<col_size:
                box+=board[i][j+idx]
                idx+=1
            if(box==10):
                answers.append([[i,j], [i,j+idx-1]])
                for r in range(idx):
                    board[i][j+r] = 0
                continue
            
            box = 0
            idx1 = 1
            idx2 = 1
            while box!=10 and i+idx1<row_size:
                idx2 = 1
                box = 0
                while box<10 and j+idx2<col_size:
                    box=sum(sum(row[j:j+idx2+1]) for row in board[i:i+idx1+1])
                    idx2+=1
                idx1+=1

            if(box==10):
                answers.append([[i,j], [i+idx1-1,j+idx2-1]])
                for r in range(idx1):
                    for c in range(idx2):
                        board[i+r][j+c] = 0
                continue

    return board, answers

In [223]:
while True:
    try:
        board = driver.find_element(By.XPATH, table)

        rows = board.find_elements(By.CLASS_NAME, 'row')
        all_cells = []

        arr = []

        for row in rows:
            all_cells.append(row.find_elements(By.CLASS_NAME, 'cell'))


        for row_cells in all_cells:
            r = []
            for cell in row_cells:
                r.append(cell.text)
            arr.append(r)

        arr = [[int(c) for c in row] for row in arr]
        print(arr)

        arr, answers = solve_routine(arr)

        while not len(answers) == 0:
            # print(len(answers))
            for [x1,y1], [x2,y2] in answers:
                all_cells[x1][y1].click()
                all_cells[x2][y2].click()

            arr, answers = solve_routine(arr)
    except:
        break

[[3, 4, 9, 7, 9, 7, 3, 2, 8, 1, 7, 9, 2, 5, 1, 3, 1], [3, 4, 3, 4, 4, 9, 2, 3, 9, 8, 8, 6, 7, 1, 3, 8, 4], [1, 6, 9, 7, 5, 1, 9, 4, 2, 7, 5, 1, 2, 7, 5, 6, 2], [6, 5, 9, 1, 9, 6, 3, 8, 4, 8, 6, 1, 6, 9, 7, 9, 6], [4, 1, 1, 5, 4, 5, 1, 5, 9, 1, 2, 7, 5, 7, 1, 6, 7], [4, 4, 9, 3, 7, 2, 7, 2, 4, 6, 4, 8, 1, 6, 6, 3, 6], [3, 5, 1, 2, 7, 9, 6, 9, 1, 2, 2, 8, 4, 2, 6, 3, 8], [6, 3, 7, 8, 5, 3, 1, 3, 3, 6, 8, 1, 1, 4, 3, 4, 3], [7, 6, 5, 1, 4, 3, 8, 9, 3, 3, 6, 8, 7, 7, 6, 5, 5], [3, 1, 9, 8, 4, 3, 3, 2, 2, 4, 2, 7, 2, 8, 9, 8, 2]]
[[5, 7, 7, 9, 6, 4, 5, 9, 5, 2, 5, 4, 7, 8, 6, 7, 8], [3, 3, 7, 3, 5, 5, 1, 2, 1, 9, 4, 3, 3, 9, 9, 8, 1], [6, 3, 9, 7, 6, 6, 7, 6, 7, 2, 6, 3, 4, 6, 5, 2, 8], [6, 7, 3, 6, 4, 2, 6, 2, 2, 7, 8, 2, 2, 7, 2, 6, 3], [7, 6, 4, 4, 9, 8, 7, 2, 7, 9, 8, 6, 1, 6, 6, 1, 7], [8, 6, 6, 2, 3, 5, 6, 2, 3, 6, 5, 9, 4, 9, 7, 1, 8], [3, 7, 8, 4, 5, 2, 9, 5, 4, 3, 8, 1, 1, 9, 5, 2, 4], [7, 9, 9, 8, 5, 5, 2, 8, 8, 9, 8, 6, 8, 5, 7, 6, 1], [1, 3, 3, 6, 1, 5, 8, 7, 4, 5, 8, 8, 5, 3, 5